# Enhanced Marine Science Object Detection
## System Information and Configuration

In [1]:
import platform
import psutil
import os
from datetime import datetime
import json
import socket

class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

def get_system_info():
    system_info = {
        'timestamp': datetime.now().isoformat(),
        'architecture': platform.machine(),
        'processor': platform.processor(),
        'platform': platform.platform(),
        'python_version': platform.python_version(),
        'hostname': socket.gethostname(),
        'total_memory_gb': round(psutil.virtual_memory().total / (1024**3), 2),
        'available_memory_gb': round(psutil.virtual_memory().available / (1024**3), 2),
        'cpu_count': psutil.cpu_count(),
        'cpu_percent': psutil.cpu_percent(interval=1),
        'docker_container_id': os.environ.get('HOSTNAME', 'Not in container')
    }
    return system_info

# Get and display system information
system_info = get_system_info()
print("=== System Information ===")
for key, value in system_info.items():
    print(f"{key}: {value}")

# Save system information to file
with open('system_info.json', 'w') as f:
    json.dump(system_info, f, indent=4, cls=DateTimeEncoder)

=== System Information ===
timestamp: 2025-05-29T04:27:42.950788
architecture: aarch64
processor: aarch64
platform: Linux-5.10.104-tegra-aarch64-with-glibc2.29
python_version: 3.8.10
hostname: e60197f29742
total_memory_gb: 61.32
available_memory_gb: 47.64
cpu_count: 12
cpu_percent: 0.7
docker_container_id: e60197f29742


In [2]:
# Resource Check and Error Handling
def check_system_resources(min_memory_gb=4, min_cpu_cores=2):
    try:
        memory_gb = psutil.virtual_memory().available / (1024**3)
        cpu_cores = psutil.cpu_count()
        
        issues = []
        if memory_gb < min_memory_gb:
            issues.append(f"Warning: Available memory ({memory_gb:.1f}GB) is below recommended minimum ({min_memory_gb}GB)")
        if cpu_cores < min_cpu_cores:
            issues.append(f"Warning: Available CPU cores ({cpu_cores}) is below recommended minimum ({min_cpu_cores})")
        
        if issues:
            print("\nResource Check Warnings:")
            for issue in issues:
                print(f"- {issue}")
            print("\nRecommendations:")
            print("- Close unnecessary applications")
            print("- Consider reducing batch size or image resolution")
            return False
        else:
            print("\nSystem resources meet minimum requirements")
            return True
    except Exception as e:
        print(f"Error checking system resources: {str(e)}")
        return False

# Perform resource check
check_system_resources()


System resources meet minimum requirements


True

In [3]:
# Import required libraries with error handling
def import_required_libraries():
    try:
        global cv2, np, Image, display, clear_output, tqdm, InferenceHTTPClient
        from inference_sdk import InferenceHTTPClient
        import cv2
        import numpy as np
        from IPython.display import Image, display, clear_output
        from tqdm.notebook import tqdm
        print("All required libraries successfully imported")
        return True
    except ImportError as e:
        print(f"Error importing required libraries: {str(e)}")
        print("\nPlease install missing libraries using:")
        print("pip install inference-sdk opencv-python numpy ipython tqdm")
        return False

# Import libraries
if not import_required_libraries():
    raise SystemExit("Required libraries not available")

All required libraries successfully imported


## Video Processing Configuration

In [4]:
# Enhanced Configuration Class
class VideoConfig:
    def __init__(self, source_path, model_id, confidence=0.3, fps_limit=10):
        self.source_path = source_path
        self.model_id = model_id
        self.confidence = confidence
        self.fps_limit = fps_limit
        self.start_time = None
        self.end_time = None
        self.processing_stats = {
            'frames_processed': 0,
            'total_detections': 0,
            'avg_confidence': 0,
            'fps': 0,
            'system_load': []
        }

    def to_dict(self):
        return {
            'source_path': self.source_path,
            'model_id': self.model_id,
            'confidence': self.confidence,
            'fps_limit': self.fps_limit,
            'start_time': self.start_time.isoformat() if self.start_time else None,
            'end_time': self.end_time.isoformat() if self.end_time else None,
            'processing_stats': self.processing_stats
        }

# Initialize configuration
config = VideoConfig(
    source_path="test_diver_video.mp4",
    model_id="fish-scuba-project/2",
    confidence=0.1,
    fps_limit=5
)

print("Configuration initialized with:")
for key, value in vars(config).items():
    print(f"{key}: {value}")

Configuration initialized with:
source_path: test_diver_video.mp4
model_id: fish-scuba-project/2
confidence: 0.1
fps_limit: 5
start_time: None
end_time: None
processing_stats: {'frames_processed': 0, 'total_detections': 0, 'avg_confidence': 0, 'fps': 0, 'system_load': []}


In [5]:
# Enhanced draw_prediction function with timestamp and stats
def draw_prediction(image, prediction, stats):
    h, w = image.shape[:2]
    x = prediction.get('x', 0.5)
    y = prediction.get('y', 0.5)
    width = prediction.get('width', 0)
    height = prediction.get('height', 0)
    
    x1 = int((x - width/2) * w)
    y1 = int((y - height/2) * h)
    x2 = int((x + width/2) * w)
    y2 = int((y + height/2) * h)
    
    x1, y1 = max(0, min(w-1, x1)), max(0, min(h-1, y1))
    x2, y2 = max(0, min(w-1, x2)), max(0, min(h-1, y2))
    
    if x2 <= x1 or y2 <= y1:
        return
    
    class_name = prediction.get('class', 'unknown')
    confidence = prediction.get('confidence', 0)
    
    color = (0, 255, 0) if confidence > 0.7 else (0, 255, 255) if confidence > 0.4 else (0, 165, 255)
    
    # Draw bounding box and label
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 4)
    
    # Add timestamp and stats
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cv2.putText(image, timestamp, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Add processing stats
    stats_text = f"FPS: {stats['fps']:.1f} | CPU: {stats['cpu_percent']}% | Mem: {stats['memory_percent']}%"
    cv2.putText(image, stats_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return image

In [6]:
# Enhanced video processing function
def process_video(config):
    config.start_time = datetime.now()
    results_data = []
    
    try:
        cap = cv2.VideoCapture(config.source_path)
        if not cap.isOpened():
            raise Exception(f"Could not open video source {config.source_path}")
        
        # Video properties
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        with tqdm(total=total_frames, desc="Processing video") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Get system stats
                stats = {
                    'timestamp': datetime.now().isoformat(),
                    'cpu_percent': psutil.cpu_percent(),
                    'memory_percent': psutil.virtual_memory().percent,
                    'fps': fps
                }
                
                # Process frame and collect results
                try:
                    result = CLIENT.infer(frame, model_id=config.model_id)
                    
                    # Store results
                    frame_result = {
                        'frame_number': config.processing_stats['frames_processed'],
                        'timestamp': stats['timestamp'],
                        'predictions': result.get('predictions', []),
                        'system_stats': stats
                    }
                    results_data.append(frame_result)
                    
                    # Update processing stats
                    config.processing_stats['frames_processed'] += 1
                    if 'predictions' in result:
                        config.processing_stats['total_detections'] += len(result['predictions'])
                        
                except Exception as e:
                    print(f"Error processing frame: {str(e)}")
                
                pbar.update(1)
        
    except Exception as e:
        print(f"Error during video processing: {str(e)}")
    finally:
        config.end_time = datetime.now()
        cap.release()
        
        # Save results to file
        results_summary = {
            'system_info': get_system_info(),
            'config': config.to_dict(),
            'processing_stats': config.processing_stats,
            'frame_results': results_data
        }
        
        with open('processing_results.json', 'w') as f:
            json.dump(results_summary, f, indent=4, cls=DateTimeEncoder)
        
        print("\nProcessing Summary:")
        print(f"Total frames processed: {config.processing_stats['frames_processed']}")
        print(f"Total detections: {config.processing_stats['total_detections']}")
        print(f"Processing time: {config.end_time - config.start_time}")

# Initialize client and run processing
CLIENT = InferenceHTTPClient(
    api_url="http://inference-server:9001",
    api_key=os.environ.get('ROBOFLOW_API_KEY')
)

# Run video processing
process_video(config)

Processing video:   0%|          | 0/250 [00:00<?, ?it/s]


Processing Summary:
Total frames processed: 250
Total detections: 0
Processing time: 0:00:14.804008
